![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [81]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Import any additional modules and start coding below
df = pd.read_csv('rental_info.csv')

Creamos primero las características pedidas.

In [82]:
df[['return_date','rental_date']] = df[['return_date','rental_date']].apply(pd.to_datetime)
df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days

In [83]:
df['deleted_scenes'] = df['special_features'].apply(lambda x : int("Deleted Scenes" in x))
df['behind_the_scenes'] = df['special_features'].apply(lambda x : int("Behind the Scenes" in x))

In [84]:
unwanted_columns = ['rental_date', 'return_date', 'special_features', 'rental_length_days']
X = df.drop(columns = unwanted_columns)
y = df['rental_length_days']

Separamos datos en conjunto de entrenamiento y conjunto de prueba.

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 9)

Probamos ahora con un modelo de regresión lineal.

In [86]:
lr = LinearRegression()
lr.fit(X_train, y_train)
predict_lr = lr.predict(X_test)

In [87]:
print(MSE(y_test, predict_lr))

2.9417238646975883


Intentaremos con un modelo de regresión con arboles de decisión. Probamos con distintos parámetros para encontrar un buen modelo.

In [88]:
dt_reg = DecisionTreeRegressor(random_state = 9)

# Create the parameter grid
param_grid = {'max_depth': [4, 6, 8, 10, 12, None], 'max_features': ['sqrt', None]} 

# Create a GridSearchCV object
grid_dt_reg = GridSearchCV(
    estimator=dt_reg,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    n_jobs=4,
    cv=5,
    refit=True, return_train_score=True)

In [89]:
grid_dt_reg.fit(X_train, y_train)
best_dt = grid_dt_reg.best_estimator_

predict_dt = best_dt.predict(X_test)
print(MSE(y_test, predict_dt))

2.1012671682443953


Realizamos la misma estrategia pero con random forests.

In [90]:
rf_reg = RandomForestRegressor(random_state = 9)

# Create the parameter grid
param_grid = {'max_depth': [60, 70, 80, 90, 100, 110], 'max_features': ['sqrt', None]} 

# Create a GridSearchCV object
grid_rf_reg = GridSearchCV(
    estimator=rf_reg,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    n_jobs=4,
    cv=5,
    refit=True, return_train_score=True)

In [91]:
grid_rf_reg.fit(X_train, y_train)
best_rf = grid_rf_reg.best_estimator_

predict_rf = best_rf.predict(X_test)
print(MSE(y_test, predict_rf))

1.9823041517415843


En este caso, el mejor modelo obtenido fue best_rf.

In [92]:
best_model = best_rf
best_mse = MSE(y_test, predict_rf)